In [8]:
import geopandas as gpd

# Especifica la ruta al archivo shapefile (reemplaza 'ruta/al/archivo.shp' con la ruta de tu archivo)
archivo_shapefile = '../../_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp'

# Utiliza la función read_file de Geopandas para cargar el shapefile
departamentos = gpd.read_file(archivo_shapefile)

In [9]:
from shapely.ops import transform

import pyproj
transformer = pyproj.Transformer.from_crs('epsg:4326', 'esri:54009', always_xy=True)

# Define a function to apply the transformation
def apply_transform(geom):
    return transform(transformer.transform, geom)

# Apply the transformation to the geometries
departamentos['geometry'] = departamentos['geometry'].apply(apply_transform)

In [ ]:
import rasterio
import matplotlib.pyplot as plt

# Path to the TIFF file
tif_file_path = '../../_data/rasters_europa/rasters/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C11.tif'

# Since we can't access external files in this environment, this code is for demonstration purposes only
# Replace the path with the actual path when you run it in your local environment

# Open the TIFF file
with rasterio.open(tif_file_path) as src:
    # Read the number of bands
    num_bands = src.count
    
    # Read the first band data
    band1 = src.read(1)
    
    # Plot the first band
    plt.imshow(band1, cmap='gray')
    plt.colorbar()
    plt.title(f'Band 1 of TIFF file (Total bands: {num_bands})')
    plt.show()
    
    # Output the number of bands
    print(f"The TIFF file has {num_bands} bands.")

In [8]:
#minx, miny, maxx, maxy = departamentos.total_bounds

In [10]:
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats

raster_paths =['../../_data/rasters_europa/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C10\GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C10.tif',
'../../_data/rasters_europa/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C11 (2)\GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C11.tif',
'../../_data/rasters_europa/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C12\GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C12.tif',
'../../_data/rasters_europa/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10 (1)\GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif',
'../../_data/rasters_europa/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11 (1)\GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif',
'../../_data/rasters_europa/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12\GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12.tif',
'../../_data/rasters_europa/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11 (1)\GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11.tif',
'../../_data/rasters_europa/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12\GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12.tif',
'../../_data/rasters_europa/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C11 (1)\GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C11.tif',
'../../_data/rasters_europa/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12 (1)\GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12.tif']

# Initialize an empty list to store results
all_stats = []

# Itera sobre las rutas de los archivos raster
for raster_path in raster_paths:
    # Calcula estadísticas zonales para cada archivo raster
    stats = zonal_stats(departamentos, raster_path, stats="unique", categorical=True)
    
    # Append the results to the list
    all_stats.append(stats)

# Concatenate all results into a final DataFrame
stats_df = pd.concat([pd.DataFrame(stats) for stats in all_stats], ignore_index=True)

df1 = pd.concat([departamentos, stats_df], axis = 1)

MemoryError: Unable to allocate 7.92 GiB for an array with shape (107123, 79400) and data type bool

In [ ]:
df1.fillna(0, inplace=True)

In [ ]:
pixel_area = 100
df1['polygon_area'] = df1['geometry'].apply(lambda x: x.area)
for category in range(1, 16):
    category_str = str(category)
    if category_str not in df1.columns:
        df1[category_str] = 0
        df1[f'MSZ_{category}_coverage'] = (df1[category_str] * pixel_area / df1['polygon_area']) * 100
     

In [ ]:
print(df1.columns) # Columns for each cathegory of MSZ

In [ ]:
# Choropleth Map

import mapclassify

for i in range(1, 16):
    column_name = f'MSZ_{i}_coverage'
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    df1.plot(
        column=column_name,
        cmap='OrRd',
        legend=True,
        scheme='quantiles',
        ax=ax
    )

    # Manually configure the legend
    leg = ax.get_legend()
    leg.set_title(f'MSZ {i}')
    leg.set_bbox_to_anchor((1.5, 0.5))

    plt.show()